# INDIGO driver development using ```pyndigo```

An INDIGO driver is a software piece that interacts with hardware and encapsulates the functionality of the **device** (or devices) by means of some **properties**. You can think of **properties** as containers for information. Properties are comoposed of one or more **items**. The property items can be of different natures:

* **Numbers**. For example: *3.1415*, *16*, *34º 25' 33''*
* **Texts**. For example: *This is a text*
* **Switches**. To select among a set of alternatives. For example: *Open* and *Closed*
* **Lights**. Flags that can be in one of 4 different predefined states: *Ok*, *Alert*, *Busy* and *Idle*.
* **BLOBs**. Binary information, for example the binary data for the pixels in a FITS image.

The properties are the only way that **clients** (that is, the software that want to interact with the devices) obtain information about the devices and can interact with them. In the following we will show how to construct a INDIGO driver using the ```pyndigo``` library.

## Creating our driver

INDIGO Drivers are executable files that are usually executed through the ```indigo_server``` service. If you have not installed INDIGO in your machine you must do it in order to have the ```indigo_server``` executable. [INDIGO downloads](https://www.indigo-astronomy.org/downloads.html).

```indigo_server``` can be used to launch as many drivers as needed. Many standard drivers come with the regular distribution of INDIGO. For example, if you want to start de INDIGO server to work with a CCD simulator and mount simulator drivers you have to run the followwing command in your shell:

```
> indigo_server indigo_ccd_simulator indigo_mount_simulator
```

Once you have launched your INDIGO server you can connect to it using any INDIGO client. For example, you can use the INDIGO Control Panel (that can also be installed in the [INDIGO downloads](https://www.indigo-astronomy.org/downloads.html)).

Our driver will be an executable python file. For example we can call it ```myDriver.py``` and we will write the code for our driver (described in the following in that file).

The first thing to do is declare the file executable. To do so we will create our file ```myDriver.py``` with the following content in your favourite text editor:

```python
#!/usr/bin/env python
```

That line indicates the operating system the the script in our file is Python code. Moreover, we have to give executable permissions to our file by executing the following command on our shell:

```
> chmod +x myDriver.py
```

Then we are going to code a **device** with its own properties. To do so, we will extend the ```PyndigoDriverDevice``` class:

```python
from pyndigo.driver import *

class FirstExampleDevice(PyndigoDriverDevice):
    def __init__(self, driver):
        super().__init__(driver, "First Example Device")
```

the ```__init__``` method of the class is the code that will be executed when the device is created. There you can define properties of the device and so on. However, until now the only thing that we are doing is specifying the name of our device as ```First Example Device``` and keeping track of the driver that will include our device.

To have a quite minimal driver example we have to create the driver itself and attach the device to it:

```
dr = PyndigoDriver("My driver")
dev = FirstExampleDevice(dr)
dr.read()
```

The first line creates a driver object, the second one instantiates de device we have previously defined (and attaching it to the driver) and the last line starts the communication between de INDIGO server and the driver. Without that line the interaction between the server and the driver will not work.

At this point we have a minimal example of driver. However it does nothing as it does not define any property. If we launch de INDIGO server with our executable file:

```
> indigo_server -i ./myDriver.py
```

and we connect with our INDIGO client we'll see the server running, but we will see no trace of our driver:

![INDIGO client. Driver not showing](imgs/driverTutorial01.png)



## Defining properties

Its time to define our first property for the driver. Lets include the following code into our constructor in the ```FirstExampleDevice``` class:

```python
class FirstExampleDevice(PyndigoDriverDevice):
    def __init__(self, driver):
        super().__init__(driver, "First Example Device")

        prop = self.add_text_property(
            "TEST_TEXT",
            STATE_OK,
            PERM_RO,
            label="First text property",
            group="Test Properties",
        )
        prop.add_text("TEXT_1", "Hello World", label="First text item")
```

We are defining one Text property that is named ```TEST_TEXT```. That property is in **OK state** (```STATE_OK```, which usually means that its contents are valid), is **Read Only** (```PERM_RO```, thus clients cannot change its value), has a label ```First text property``` (which is the text that clients will usually render as its name) and belongs to a group of properties called ```Test Properties```. The group of a property is usually used by the clients to visually organize the different properties in a device.

Finally, we are adding one text item for the property called ```TEXT_1``` with a particular value of ```Hello World``` and a label ```First text item```. Remember that properties can have more than one item.

If we re-launch or INDIGO server and connect to it we will see our driver working with that particular text property and item:

![INDIGO client with our driver working](./imgs/driverTutorial02.png)

Lets add a second property to our driver. In this case, a numerical one with two different number items:

```python
        prop = self.add_number_property(
            "TEST_NUMBER",
            STATE_OK,
            PERM_RW,
            label="First number property",
            group="Test Properties",
        )
        prop.add_number("SPEED", "10", number_format="%1.0f", min_value=0, max_value=100, step=1, label="Speed")
        prop.add_number("ACCEL", "1", number_format="%1.0f", min_value=0, max_value=100, step=1, label="Acceleration")
```

In this case the property is called ```TEST_NUMBER``` is a **Read/Write** property, that is, the property can be read and written by the clients and has two numerical items called ```SPEED``` and ```ACCEL```. Notice that numerical items have a minimum, a maximum and a step values that should be defined. Moreover, thay also have a numebr format that should be written as printf - C-style. In this case they are floating numbers but not decimal places should be shown. Please note that the number format can also be a number in sexagesimal form. Check the INDI Whitepaper for more information about it. If we re-launch the server and connect with our client we should see this new property:

![INDIGO client with new numerical property](./imgs/driverTutorial03.png)

Please note that as the property is Read/Write the client allows changing its values (extra fields to input the desired value and ```Reset``` and ```Set``` buttons.